In [ ]:
import pandas as pd
traffic_data=pd.read_csv('Traffic.csv')
traffic_data

,Time,Date,Day of the week,CarCount,BikeCount,BusCount,TruckCount,Total,Traffic Situation
0,12:00:00 AM,10,Tuesday,31,0,4,4,39,low
1,12:15:00 AM,10,Tuesday,49,0,3,3,55,low
2,12:30:00 AM,10,Tuesday,46,0,3,6,55,low
3,12:45:00 AM,10,Tuesday,51,0,2,5,58,low
4,1:00:00 AM,10,Tuesday,57,6,15,16,94,normal
...,...,...,...,...,...,...,...,...,...
2971,10:45:00 PM,9,Thursday,16,3,1,36,56,normal
2972,11:00:00 PM,9,Thursday,11,0,1,30,42,normal
2973,11:15:00 PM,9,Thursday,15,4,1,25,45,normal
2974,11:30:00 PM,9,Thursday,16,5,0,27,48,normal


In [ ]:
def preprocess_traffic_data(traffic_data):
    # Gabungkan kolom Date dan Time menjadi satu kolom datetime
    traffic_data["Datetime"] = pd.to_datetime(
        traffic_data["Date"].astype(str) + " " + traffic_data["Time"],
        format="%d %I:%M:%S %p",  # Format sesuai contoh: "10 12:15:00 AM"
        errors="coerce"  # Menangani error parsing
    )
    # Ekstrak jam dan menit dari kolom Datetime
    traffic_data["Hour"] = traffic_data["Datetime"].dt.hour
    traffic_data["Minute"] = traffic_data["Datetime"].dt.minute

    situation_mapping = {"low": 0, "normal": 1, "high": 2, "heavy": 3}
    traffic_data["Traffic_Situation_Code"] = traffic_data["Traffic Situation"].map(situation_mapping)
    return traffic_data

In [ ]:
print(traffic_data.head())  # Periksa 5 baris pertama data


          Time  Date Day of the week  CarCount  BikeCount  BusCount  \
0  12:00:00 AM    10         Tuesday        31          0         4   
1  12:15:00 AM    10         Tuesday        49          0         3   
2  12:30:00 AM    10         Tuesday        46          0         3   
3  12:45:00 AM    10         Tuesday        51          0         2   
4   1:00:00 AM    10         Tuesday        57          6        15   

   TruckCount  Total Traffic Situation  
0           4     39               low  
1           3     55               low  
2           6     55               low  
3           5     58               low  
4          16     94            normal  


In [ ]:
print(traffic_data.info())  # Periksa struktur data dan kolom yang ada


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2976 entries, 0 to 2975
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Time               2976 non-null   object
 1   Date               2976 non-null   int64 
 2   Day of the week    2976 non-null   object
 3   CarCount           2976 non-null   int64 
 4   BikeCount          2976 non-null   int64 
 5   BusCount           2976 non-null   int64 
 6   TruckCount         2976 non-null   int64 
 7   Total              2976 non-null   int64 
 8   Traffic Situation  2976 non-null   object
dtypes: int64(6), object(3)
memory usage: 209.4+ KB
None


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
# Fungsi untuk menghitung jarak Euclidean
def euclidean_distance(x1, x2):
    return np.sqrt(np.sum((x1 - x2) ** 2))

In [ ]:
# Fungsi KNN manual
def knn_manual(X_train, y_train, X_test, k=3):
    predictions = []
    for test_point in X_test:
        # Hitung jarak dari setiap titik data dalam data training
        distances = [euclidean_distance(test_point, train_point) for train_point in X_train]

        # Urutkan berdasarkan jarak dan ambil k tetangga terdekat
        k_indices = np.argsort(distances)[:k]
        k_nearest_labels = [y_train[i] for i in k_indices]

        # Tentukan label yang paling umum
        most_common = max(set(k_nearest_labels), key=k_nearest_labels.count)
        predictions.append(most_common)
    return predictions

In [ ]:
# Data Preprocessing
traffic_data_preprocessed = preprocess_traffic_data(traffic_data)

In [ ]:
# Extract features and target
X = traffic_data_preprocessed[["Hour", "Minute"]].values
y = traffic_data_preprocessed["Traffic_Situation_Code"].values

In [ ]:
X

array([[ 0,  0],
       [ 0, 15],
       [ 0, 30],
       ...,
       [23, 15],
       [23, 30],
       [23, 45]], dtype=int32)

In [ ]:
y

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
# Split data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
for k in range(1, 21):  # Coba nilai k dari 1 hingga 20
    y_pred = knn_manual(X_train, y_train, X_test, k=k)
    accuracy = np.mean(y_pred == y_test)
    print(f"k = {k}, Accuracy = {accuracy:.2f}")


k = 1, Accuracy = 0.58
k = 2, Accuracy = 0.53
k = 3, Accuracy = 0.65
k = 4, Accuracy = 0.65
k = 5, Accuracy = 0.66
k = 6, Accuracy = 0.67
k = 7, Accuracy = 0.67
k = 8, Accuracy = 0.67
k = 9, Accuracy = 0.69
k = 10, Accuracy = 0.68
k = 11, Accuracy = 0.69
k = 12, Accuracy = 0.68
k = 13, Accuracy = 0.69
k = 14, Accuracy = 0.68
k = 15, Accuracy = 0.69
k = 16, Accuracy = 0.69
k = 17, Accuracy = 0.69
k = 18, Accuracy = 0.68
k = 19, Accuracy = 0.68
k = 20, Accuracy = 0.68


In [ ]:
# KNN Prediction (manual)
k = 13
y_pred = knn_manual(X_train, y_train, X_test, k=k)

In [ ]:
X_train

array([[ 5, 15],
       [ 7, 15],
       [ 7,  0],
       ...,
       [18, 30],
       [11, 30],
       [23,  0]], dtype=int32)

In [ ]:
y_train

array([0, 3, 3, ..., 3, 1, 1])

In [ ]:
# Evaluasi Akurasi
accuracy = np.mean(y_pred == y_test)
print(f"Akurasi model KNN manual: {accuracy:.2f}")

Akurasi model KNN manual: 0.69


In [ ]:
# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        66
           1       0.74      0.85      0.79       340
           2       0.29      0.22      0.25        63
           3       0.68      0.83      0.75       127

    accuracy                           0.69       596
   macro avg       0.43      0.48      0.45       596
weighted avg       0.60      0.69      0.64       596



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
situation_decoding = {0: "low", 1: "normal", 2: "high", 3: "heavy"}

In [ ]:
# Contoh Prediksi
test_hour = 8  # Jam contoh
test_minute = 30  # Menit contoh
test_input = np.array([[test_hour, test_minute]])

predicted_code = knn_manual(X_train, y_train, test_input, k=k)[0]
predicted_situation = situation_decoding[predicted_code]
print(f"Prediksi situasi lalu lintas pukul {test_hour}:{test_minute} --> {predicted_situation}")


Prediksi situasi lalu lintas pukul 8:30 --> heavy
